In [44]:
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
from pynytimes import NYTAPI
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
import numpy as np
import time
from tqdm import tqdm

load_dotenv()

True

In [32]:
API_KEY = os.getenv("NYT_KEY") or ""
nyt = NYTAPI(API_KEY, parse_dates=True)

In [33]:
# Get data
# NOTE: Will need to be tweaked once I'm not in a car going really fast

# Grrr, can't download stuff no, come back to it
data = nyt.archive_metadata(
    date = datetime(2001, 4, 1)
)

In [30]:
model = SentenceTransformer('sentence-transformers/sentence-t5-base')

for thing in data[:100]:
    print(thing["abstract"])
    print(thing["lead_paragraph"])
    print(thing["headline"]["main"])
    print(thing.keys())
    print()

Ernest Auerbach letter warns on avoiding street food in Mexico City (S)
To the Editor: Having lived in Mexico City for three and a half years, I strongly disagree with a comment by Tim Weiner: ''It is difficult to get a bad meal in a city with some of the world's best and cheapest street food and too many restaurants to count.''
Mexico City
dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

Peter Verniero could have gone back to Morristown to golf and spend clients' money, a moderately big deal still addressed as ''general'' by the occasional lawyer or waiter.    It's not as good as being attorney general, but it sounds better than being the Clarence Thomas of the New Jersey Supreme Court.  
Peter Verniero could have gone back to Morristown to golf and spend clients' money, a mode

In [34]:
embeds = model.encode([thing["headline"]["main"] for thing in data[:10]])
print(len(embeds[0]))

768


In [45]:
EMBED_LENGTH = 768
NUM_EMBEDS = 3
model = SentenceTransformer('sentence-transformers/sentence-t5-base')

# Fetching the dates
def main_loop():
    aix = AnnoyIndex(EMBED_LENGTH * NUM_EMBEDS, 'angular')
    count = 0
    dates = []
    date = datetime(2001, 4, 29)
    while date < datetime.now() - timedelta(days=365 * 10):
        dates.append(date)
        date += timedelta(days=1)
        break
    for ix, date in enumerate(dates):
        print(f"{ix + 1}/{len(dates)}")
        start_time = datetime.now()
        data = nyt.archive_metadata(date)
        for article in tqdm(data):
            abs_emb = model.encode(article["abstract"])
            par_emb = model.encode(article["lead_paragraph"])
            head_emb = model.encode(article["headline"]["main"])
            concat = np.concatenate([abs_emb, par_emb, head_emb])
            aix.add_item(count, concat)
            count += 1
        date += timedelta(days=1)
        end_time = datetime.now()
        diff = end_time - start_time
        if diff.seconds < 5:
            time.sleep(diff.seconds)
    aix.build(36, -1)
    aix.save("articles.tree")
    
main_loop()

  0%|          | 0/1 [00:17<?, ?it/s]


KeyboardInterrupt: 